In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nsl-kdd-augmented/smote_augmented.csv
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Novelty: Center-Loss Expansion Specialist
# ===========================================
class MLAR_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes, embed_dim=2048):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Linear(input_dim, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.SiLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Linear(embed_dim, num_classes)
        # Learnable Anchors (Centers) for each class
        self.centers = nn.Parameter(torch.randn(num_classes, embed_dim))

    def forward(self, x):
        features = self.feat(x)
        logits = self.classifier(features)
        return logits, features

# ===========================================
# 2️⃣ Custom Loss: Soft-Margin + Center Penalty
# ===========================================
def mlar_criterion(logits, features, targets, centers, weight, lambd=0.01):
    # Standard Weighted Cross Entropy
    ce_loss = F.cross_entropy(logits, targets, weight=weight)
    
    # Center Loss: Forces features to cluster around class anchors
    batch_centers = centers[targets]
    center_loss = F.mse_loss(features, batch_centers)
    
    return ce_loss + lambd * center_loss

# ===========================================
# 3️⃣ Training the Specialist
# ===========================================
model_sp = MLAR_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_sp.parameters(), lr=3e-4, weight_decay=1e-2)

# Specific weights for classes with poor precision/recall balance
sp_weights = torch.ones(num_classes).to(device)
for idx in hard_indices: sp_weights[idx] = 40.0 # Extreme focus

for epoch in range(30):
    model_sp.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, feats = model_sp(xb)
        loss = mlar_criterion(logits, feats, yb, model_sp.centers, sp_weights)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Expert: XGBoost (Optimized for Macro F1)
# ===========================================
expert = XGBClassifier(tree_method='hist', device='cuda', 
                       n_estimators=300, max_depth=10, 
                       learning_rate=0.05, colsample_bytree=0.8)
expert.fit(X_train_proc, y_train_enc)

# ===========================================
# 5️⃣ Final Inference: Dynamic Bayesian Fusion
# ===========================================
def final_q1_fusion(X_proc):
    model_sp.eval()
    with torch.no_grad():
        logits, _ = model_sp(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_sp = torch.softmax(logits * 1.2, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    final_preds = []
    for i in range(len(X_proc)):
        # Calculate confidence gap
        conf_sp = np.max(probs_sp[i])
        conf_ex = np.max(probs_ex[i])
        
        sp_choice = np.argmax(probs_sp[i])
        
        # If Specialist is confident in a HARD class, it wins
        if sp_choice in hard_indices and probs_sp[i][sp_choice] > 0.35:
            final_preds.append(sp_choice)
        else:
            final_preds.append(np.argmax(probs_ex[i]))
            
    return np.array(final_preds)

final_preds = final_q1_fusion(X_test_proc)
unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))

                 precision    recall  f1-score   support

           back       1.00      1.00      1.00       359
buffer_overflow       0.34      0.55      0.42        20
      ftp_write       0.02      0.67      0.04         3
   guess_passwd       0.99      0.27      0.42      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.98      0.99       141
           land       1.00      0.43      0.60         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      1.00      1.00        73
         normal       0.87      0.97      0.92      9711
           perl       0.50      0.50      0.50         2
            phf       1.00      0.50      0.67         2
            pod       0.69      0.88      0.77        41
      portsweep       0.79      0.95      0.86       157
        rootkit       0.01    